In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [31]:
d=dict()
for i in range(1,17):
    d1=dict()
    for mode in ['lokaralpha','womenar','nominated1to12']:
        soup = BeautifulSoup(requests.get(f"https://loksabha.nic.in/Members/{mode}.aspx?lsno={i}&tab=0").text, 'lxml')
        row_list = soup.find("table", {"class": "member_list_table"}).find_all("tr")[1:]
        l=list()
        for row in row_list:
            l1=list()
            for col in row.find_all("td"):
                l1.append(re.sub(r" {1,}"," ",col.text.strip()))
            l.append(l1)
        d1[mode]=l
    d[i]=d1


In [69]:
final_df=pd.DataFrame(columns=["name",'party','Constituency & State','gender','is_nominated',"nth loksabha"])

mode =['lokaralpha','womenar','nominated1to12']
for key,values in d.items():
    x,y,z=pd.DataFrame(values[mode[0]]),pd.DataFrame(values[mode[1]]),pd.DataFrame(values[mode[2]])
    x["gender"]="male"
    x['is_nominated']=0
    l1=list(set(x[1]) & set(y[1]))
    l2=list(set(x[1]) & set(z[1]))
    if not(len(l1)==len(y[1]) or len(l2)==len(z[1])):
        print("error",len(l1)-len(y[1]),len(l2)-len(z[1]),i)
    x.loc[x[x[1].isin(l1)].index,'gender']="female"
    x.loc[x[x[1].isin(l2)].index,'is_nominated']=1
    x['nth loksabha']=key
    x.drop(0,axis=1,inplace=True)
    x.columns=["name",'party','Constituency & State','gender','is_nominated',"nth loksabha"]
    final_df=pd.concat([final_df,x],ignore_index=True)

In [70]:
final_df.head()

,name,party,Constituency & State,gender,is_nominated,nth loksabha
0,"Abdullabhai,Mulla Taherali Mulla",Congress,Chanda (Madhya Pradesh ),male,0,1
1,"Abdus Sattar,Shri",Congress,Kalna-Katwa (West Bengal ),male,0,1
2,"Achal Singh,Seth",Congress,Agra (Uttar Pradesh ),male,0,1
3,"Achalu,Shri Sunkam",PDF,Nalgonda-SC (Hyderabad ),male,0,1
4,"Achint Ram,Lala",Congress,Hissar (Punjab ),male,0,1


In [71]:
final_df.count()

name                    8889
party                   8889
Constituency & State    8889
gender                  8889
is_nominated            8889
nth loksabha            8889
dtype: int64

In [73]:
final_df.to_csv("1-16 loksabha members.csv")

AttributeError: module 'pandas' has no attribute 'to_csv'